In [ ]:
import time
import numpy as np
import matplotlib.pyplot as plt
from sliderule import gedi
from sliderule import earthdata
import sliderule

In [ ]:
# initialize client (notebook only processes one granule, so one node is sufficient)
gedi.init("slideruleearth.io", verbose=True)

In [ ]:
# Specify region of interest from geojson
poly_fn = 'grandmesa.geojson'
region = sliderule.toregion(poly_fn)

In [ ]:
granules = earthdata.cmr(short_name="GEDI01_B", polygon=region["poly"])

In [ ]:
# Build GEDI L4A Request Parameters
parms = {
    "poly": region["poly"],
    "degrade_flag": 0,
    "quality_flag": 1,
    "beam": 0
}

In [ ]:
# Latch Start Time
perf_start = time.perf_counter()

# Request GEDI Data
gedi01b = gedi.gedi01bp(parms, resources=['GEDI01_B_2019109210809_O01988_03_T02056_02_005_01_V002.h5'])
                        
# Latch Stop Time
perf_stop = time.perf_counter()

# Display Statistics
perf_duration = perf_stop - perf_start
print("Completed in {:.3f} seconds of wall-clock time".format(perf_duration))
print("Received {} footprints".format(gedi01b.shape[0]))
if len(gedi01b) > 0:
    print("Beams: {}".format(gedi01b["beam"].unique()))

In [ ]:
gedi01b

In [ ]:
# plot elevations
f, ax = plt.subplots(1, 2, figsize=[12,8])
ax[0].set_title("Elevation of First Bin")
ax[0].set_aspect('equal')
gedi01b.plot(ax=ax[0], column='elevation_start', cmap='inferno', s=0.1)
ax[1].set_title("Elevation of Last Bin")
ax[1].set_aspect('equal')
gedi01b.plot(ax=ax[1], column='elevation_stop', cmap='inferno', s=0.1)

In [ ]:
f, ax = plt.subplots()
i = 0
waveform = np.ma.masked_equal(gedi01b['rx_waveform'].iloc[i], 0).compressed()
elev_bins = np.linspace(gedi01b.iloc[i]['elevation_stop'], gedi01b.iloc[i]['elevation_start'], gedi01b.iloc[i]['rx_size'])
ax.plot(waveform, elev_bins)
ax.set_ylabel('Elevation (m)')
ax.set_xlabel('Amplitude')
title = f'{gedi01b.index[i]}, Beam {gedi01b.iloc[i]["beam"]}'
ax.set_title(title)

In [ ]:
f, ax = plt.subplots()
for index, val in gedi01b.iterrows():
    waveform = np.ma.masked_outside(val['rx_waveform'], 100, 500).compressed()
    elev_bins = np.linspace(val['elevation_stop'], val['elevation_start'], waveform.shape[0])
    ax.plot(waveform, elev_bins)
#ax.set_ylabel('Elevation (m)')
#ax.set_xlabel('Amplitude')